###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Departamento Nacional de Planeación

In [1]:
import pandas as pd

###Descarga del dataset desde el endpoint - archivo np8m-kdhq.csv

Contiene la informacion de viviendas de la base de datos del Sisbén IV, del  Departamento Nacional de Planeación (DNP). Corresponden a  muestras anonimizadas de las encuestas del Sisbén. El enlace a la fuente donde puede consultarse la metadata y el diccionario de datos entre otros aspectos es: https://anda.dnp.gov.co/index.php/catalog/150/datafile/F17 y otros documentos sobre el sisben IV se pueden consultar en https://www.sisben.gov.co/Paginas/documentos-sisben.html

In [3]:
sisben_IV_viviendas_DNP = pd.read_csv("https://www.datos.gov.co/resource/np8m-kdhq.csv?$limit=3000000")
sisben_IV_viviendas_DNP.head(3)

,cod_mpio,zona,llave,corte,fex,viv001,viv002,viv003,viv004,viv005,viv006,viv007,viv008,viv009,viv010
0,5001,1,1,SIV_2022,761.196393,2,1,2,1,1,1,1,1,4,1
1,5001,2,2,SIV_2022,23.954887,1,1,3,1,1,2,2,1,3,1
2,5001,1,3,SIV_2022,761.196393,2,1,2,1,1,1,1,1,4,2


###Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [4]:
sisben_IV_viviendas_DNP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1446237 entries, 0 to 1446236
Data columns (total 15 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   cod_mpio  1446237 non-null  int64  
 1   zona      1446237 non-null  int64  
 2   llave     1446237 non-null  int64  
 3   corte     1446237 non-null  object 
 4   fex       1446237 non-null  float64
 5   viv001    1446237 non-null  int64  
 6   viv002    1446237 non-null  int64  
 7   viv003    1446237 non-null  int64  
 8   viv004    1446237 non-null  int64  
 9   viv005    1446237 non-null  int64  
 10  viv006    1446237 non-null  int64  
 11  viv007    1446237 non-null  int64  
 12  viv008    1446237 non-null  int64  
 13  viv009    1446237 non-null  int64  
 14  viv010    1446237 non-null  int64  
dtypes: float64(1), int64(13), object(1)
memory usage: 165.5+ MB


- Descripcion de cada columna

In [5]:
column_mapping = {
    'cod_mpio': 'Código de municipio',
    'zona': 'Zona',
    'llave': 'Identificación de la vivienda',
    'corte': 'Corte de los datos',
    'fex': 'Factor de expansión',
    'viv001': 'Tipo de vivienda',
    'viv002': 'Material predominante de paredes',
    'viv003': 'Material predominante de los pisos',
    'viv004': 'Energía eléctrica',
    'viv005': 'Alcantarillado',
    'viv006': 'Gas natural domiciliario',
    'viv007': 'Recolección de basuras',
    'viv008': 'Acueducto',
    'viv009': 'Número de cuartos, incluyendo sala comedor que cuenta la vivienda',
    'viv010': 'Cuántos grupos de personas que manejan su propio presupuesto (hogares) hay en la vivienda'

}

In [6]:
columns = sisben_IV_viviendas_DNP.columns

In [7]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {sisben_IV_viviendas_DNP[col].nunique()}')

cod_mpio: 1099
zona: 2
llave: 3401
corte: 1
fex: 2179
viv001: 5
viv002: 7
viv003: 6
viv004: 2
viv005: 2
viv006: 2
viv007: 2
viv008: 2
viv009: 15
viv010: 12


In [8]:
# Obtener las columnas que no son numéricas
non_numeric_columns = sisben_IV_viviendas_DNP.select_dtypes(exclude=['int64', 'float64']).columns
print(non_numeric_columns)

Index(['corte'], dtype='object')


In [9]:
sisben_IV_viviendas_DNP[non_numeric_columns].head()

,corte
0,SIV_2022
1,SIV_2022
2,SIV_2022
3,SIV_2022
4,SIV_2022


In [10]:
# Eliminar columnas que se considera innecesarias para el proyecto
delete_columns = [ ]# Eliminando las columnas especificadas
sisben_IV_viviendas_DNP = sisben_IV_viviendas_DNP.drop(delete_columns, axis=1)

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [11]:
# Imprimir categorías únicas para columnas de tipo object
for column in non_numeric_columns:
    print(f"Categorías en la columna '{column}':")
    print(sisben_IV_viviendas_DNP[column].unique())
    print()

Categorías en la columna 'corte':
['SIV_2022']



In [12]:
# Mostrar los valores únicos de cada columna que tenga menos de 50 categorías
for col in non_numeric_columns:
  l = sisben_IV_viviendas_DNP[col].nunique()
  if l < 50:
    print(f'{col}: {sisben_IV_viviendas_DNP[col].unique()}')

corte: ['SIV_2022']


- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [13]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [14]:
# Aplicar la función a todas las columnas categóricas
sisben_IV_viviendas_DNP[non_numeric_columns] = sisben_IV_viviendas_DNP[non_numeric_columns].astype(str)
for col in non_numeric_columns:
    sisben_IV_viviendas_DNP[col] = sisben_IV_viviendas_DNP[col].apply(remove_accents_and_special_chars)

### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.

En el presente df los códigos de los municipios estan en formato int64


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [15]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


In [16]:
sisben_IV_viviendas_DNP.columns

Index(['cod_mpio', 'zona', 'llave', 'corte', 'fex', 'viv001', 'viv002',
       'viv003', 'viv004', 'viv005', 'viv006', 'viv007', 'viv008', 'viv009',
       'viv010'],
      dtype='object')

- Longitud de los códigos municipios en el df

In [17]:
# Convertir a strings
sisben_IV_viviendas_DNP["cod_mpio"] = sisben_IV_viviendas_DNP["cod_mpio"].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_mpio = sisben_IV_viviendas_DNP["cod_mpio"].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los mpios tiene longitudes de :[4 5] caracteres


In [18]:
# Contar registros por longitud
long_df = longitud_mpio.value_counts().reset_index()
long_df.rename(columns={'cod_mpio': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitud_mpio))
long_df.head()

,no_dígitos_codigo_dane,count,percentage
0,5,1260893,0.871844
1,4,185344,0.128156


In [19]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitud_mpio.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = sisben_IV_viviendas_DNP[longitud_mpio == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['cod_mpio']])
    print()

Muestra de registros con longitud 5:
     cod_mpio
1013    25320
1014    27372
1015    25805
1016    50590
1017    17001

Muestra de registros con longitud 4:
  cod_mpio
0     5001
1     5001
2     5001
3     5001
4     5001



Nota: De lo anterior se notan claras inconsistencias en la columna 'codigo_dane' del df, el 87.2 tiene 5 dígitos (que corresponde con el codigo real que es de 5 dígitos), el 12.8% tiene solamente 4 dígitos (al parecer el cero a la izquierda de los códigos se suprimió).

- Verificar si los codigo_dane de 4 dígitos corresponden a departamentos que se identifican con el 0 + 1 dígito, para validar la teoria de que al generar el dataset se les suprimió el cero a la izquierda

In [20]:
# Filtrar las filas donde 'codigo_dane' tiene 7 dígitos
filtered = sisben_IV_viviendas_DNP[sisben_IV_viviendas_DNP['cod_mpio'].str.len() == 4]

# Obtener el primer dígito de cada código y convertirlo a una serie
first_digit = filtered['cod_mpio'].str[0]
# Obtener las categorías únicas de la columna 'departamento'
first_digit.unique()

array(['5', '8'], dtype=object)

Efectivamente ANTIOQUIA Y ATLANTICO son los unicos departamentos que tienen codigo Dane que inicia con un cero + un dígito, 05 y 08 respectivamente.

  - Adicionar un cero a los codigo_dane de 4 dígitos

In [21]:
# Función que agrega un '0' a la izquierda si la longitud del string es 7
def add_zero_if_length_4(codigo):
    if len(codigo) == 4:
        return '0' + codigo
    return codigo

# Aplicar la función a la columna 'codigo'
sisben_IV_viviendas_DNP['cod_mpio'] = sisben_IV_viviendas_DNP['cod_mpio'].apply(add_zero_if_length_4)

In [22]:
# Verificar cuantos codigo_dane de 4 dígitos quedaron
len(sisben_IV_viviendas_DNP[sisben_IV_viviendas_DNP['cod_mpio'].str.len() == 4])

0

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [23]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [24]:
# Comparar listas de códigos
compare_lists(sisben_IV_viviendas_DNP['cod_mpio'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en sisben_IV_viviendas_DNP", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en sisben_IV_viviendas_DNP que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [25]:
# Adicionar columna para trazabilidad de la fuente
# Lista de tuplas
sisben_IV_viviendas_DNP['source_id'] = 110


In [26]:
#Estructura final del dataset a integrar a la base de datos
sisben_IV_viviendas_DNP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1446237 entries, 0 to 1446236
Data columns (total 16 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   cod_mpio   1446237 non-null  object 
 1   zona       1446237 non-null  int64  
 2   llave      1446237 non-null  int64  
 3   corte      1446237 non-null  object 
 4   fex        1446237 non-null  float64
 5   viv001     1446237 non-null  int64  
 6   viv002     1446237 non-null  int64  
 7   viv003     1446237 non-null  int64  
 8   viv004     1446237 non-null  int64  
 9   viv005     1446237 non-null  int64  
 10  viv006     1446237 non-null  int64  
 11  viv007     1446237 non-null  int64  
 12  viv008     1446237 non-null  int64  
 13  viv009     1446237 non-null  int64  
 14  viv010     1446237 non-null  int64  
 15  source_id  1446237 non-null  int64  
dtypes: float64(1), int64(13), object(2)
memory usage: 176.5+ MB


In [27]:
sisben_IV_viviendas_DNP.head()

,cod_mpio,zona,llave,corte,fex,viv001,viv002,viv003,viv004,viv005,viv006,viv007,viv008,viv009,viv010,source_id
0,05001,1,1,SIV_2022,761.196393,2,1,2,1,1,1,1,1,4,1,110
1,05001,2,2,SIV_2022,23.954887,1,1,3,1,1,2,2,1,3,1,110
2,05001,1,3,SIV_2022,761.196393,2,1,2,1,1,1,1,1,4,2,110
3,05001,1,4,SIV_2022,761.196393,2,1,2,1,1,1,1,1,3,1,110
4,05001,2,5,SIV_2022,23.954887,1,1,3,1,2,2,1,1,3,1,110


## Salvar en archivo csv en el drive

In [28]:
# Guardar en archivos CSV en el drive
sisben_IV_viviendas_DNP.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/sisben_IV_viviendas_DNP.csv', index=False)